In [45]:
%load_ext autoreload
%autoreload 2

import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from evolver import CrossoverType, MutationType, MatrixEvolver
from unet import UNet
from dataset_utils import PartitionType
from landcover_dataloader import LandCoverDataset, create_land_cover_dataset_from_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)  

In [51]:
params = {'batch_size': 16,
          'shuffle': False,
          'num_workers': 6}

dataset_name = "data/landcover_small"
create_land_cover_dataset_from_config(os.path.join(os.getcwd(), dataset_name))
train_loader = torch.utils.data.DataLoader(PartitionType.TRAIN, **params)
validation_loader = torch.utils.data.DataLoader(PartitionType.VALIDATION, **params)
finetuning_loader = torch.utils.data.DataLoader(PartitionType.FINETUNING, **params)
test_loader = torch.utils.data.DataLoader(PartitionType.TEST, **params)

In [35]:
# Define Model Loss and Optimizers
max_epochs = 10
report_batches = 25
net = UNet(in_channels = 4, n_classes = train_set.n_classes, depth = 4)
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(max_epochs):
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        batch_x, batch_y = data
        batch_x, batch_y = batch_x.to(device), batch_y.to(device) 
        
        optimizer.zero_grad()
        outputs = net(batch_x)

        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if i % report_batches == report_batches - 1:
            print('TRAIN: [Epoch: %d,  Batch: %d, Epoch Percent: %0.2f, Avg Loss: %3d]' % (epoch + 1, 
                                                                                           i + 1, 
                                                                                           100 * train_loader.batch_size * (i + 1) / train_set.n_samples),
                                                                                           running_loss / (report_batches * train_loader.batch_size))
            running_loss = 0.0
    
    running_loss = 0.0
    for i, data in enumerate(test_loader):
        batch_x, batch_y = data
        batch_x, batch_y = batch_x.to(device), batch_y.to(device) 
        outputs = net(batch_x)
        loss = criterion(outputs, batch_y)
        running_loss += loss.item()

    print('TEST: [Avg Batch Loss: %3d]' % (running_loss / ((i + 1) * test_loader.batch_size))

[Epoch: 1,  Batch: 10, Epoch Percent: 3.20]
[Epoch: 1,  Batch: 20, Epoch Percent: 6.40]
[Epoch: 1,  Batch: 30, Epoch Percent: 9.60]
[Epoch: 1,  Batch: 40, Epoch Percent: 12.80]
[Epoch: 1,  Batch: 50, Epoch Percent: 16.00]
[Epoch: 1,  Batch: 60, Epoch Percent: 19.20]
[Epoch: 1,  Batch: 70, Epoch Percent: 22.40]
[Epoch: 1,  Batch: 80, Epoch Percent: 25.60]
[Epoch: 1,  Batch: 90, Epoch Percent: 28.80]
[Epoch: 1,  Batch: 100, Epoch Percent: 32.00]
[Epoch: 1,  Batch: 110, Epoch Percent: 35.20]
[Epoch: 1,  Batch: 120, Epoch Percent: 38.40]
[Epoch: 1,  Batch: 130, Epoch Percent: 41.60]
[Epoch: 1,  Batch: 140, Epoch Percent: 44.80]
[Epoch: 1,  Batch: 150, Epoch Percent: 48.00]
[Epoch: 1,  Batch: 160, Epoch Percent: 51.20]
[Epoch: 1,  Batch: 170, Epoch Percent: 54.40]
[Epoch: 1,  Batch: 180, Epoch Percent: 57.60]
[Epoch: 1,  Batch: 190, Epoch Percent: 60.80]
[Epoch: 1,  Batch: 200, Epoch Percent: 64.00]
[Epoch: 1,  Batch: 210, Epoch Percent: 67.20]
[Epoch: 1,  Batch: 220, Epoch Percent: 70.40]


[Epoch: 6,  Batch: 270, Epoch Percent: 86.40]
[Epoch: 6,  Batch: 280, Epoch Percent: 89.60]
[Epoch: 6,  Batch: 290, Epoch Percent: 92.80]
[Epoch: 6,  Batch: 300, Epoch Percent: 96.00]
[Epoch: 6,  Batch: 310, Epoch Percent: 99.20]
[Epoch: 7,  Batch: 10, Epoch Percent: 3.20]
[Epoch: 7,  Batch: 20, Epoch Percent: 6.40]
[Epoch: 7,  Batch: 30, Epoch Percent: 9.60]
[Epoch: 7,  Batch: 40, Epoch Percent: 12.80]
[Epoch: 7,  Batch: 50, Epoch Percent: 16.00]
[Epoch: 7,  Batch: 60, Epoch Percent: 19.20]
[Epoch: 7,  Batch: 70, Epoch Percent: 22.40]
[Epoch: 7,  Batch: 80, Epoch Percent: 25.60]
[Epoch: 7,  Batch: 90, Epoch Percent: 28.80]
[Epoch: 7,  Batch: 100, Epoch Percent: 32.00]
[Epoch: 7,  Batch: 110, Epoch Percent: 35.20]
[Epoch: 7,  Batch: 120, Epoch Percent: 38.40]
[Epoch: 7,  Batch: 130, Epoch Percent: 41.60]
[Epoch: 7,  Batch: 140, Epoch Percent: 44.80]
[Epoch: 7,  Batch: 150, Epoch Percent: 48.00]
[Epoch: 7,  Batch: 160, Epoch Percent: 51.20]
[Epoch: 7,  Batch: 170, Epoch Percent: 54.40]


In [ ]:
# create hook in unet to set dropout on up and down layers.
# consider a set dropout layer function for up or down layer

# create hook in unet to get layer dimensions of the outputs to each layer after conv b
# scale dropout mask before applying it
# consider adding "temperature" or learning rate adjustment to mutation and cross over types.
# add cross over type best.

# finish commenting matrix evolver.
dropout_mask_evolver = MatrixEvolver([[3, 3]], CrossoverType.UNIFORM, MutationType.FLIP_BIT)

In [ ]:
# for p in net.parameters():
#     print(p.shape)